# 网络中的网络

* LeNet、AlexNet和VGG都有一个共同的设计模式
    * 通过一系列的卷积层与池化层来提取空间结构特征
    * 然后通过全连接层对特征的表征进行处理
* AlexNet和VGG对LeNet的改进主要在于如何扩大和加深（widen and deepen）这两个模块


# 网络中的网络

* one could imagine using fully-connected layers earlier in the process
    * a careless use of dense layers might give up the spatial structure of the representation entirely
* 视频：[网络中的网络（NiN）](https://www.bilibili.com/video/BV1Uv411G71b?p=1)
    * 全连接层的问题

## NiN块

* NiN的想法是在每个像素位置应用一个全连接层
* NiN块以一个普通卷积层开始，后面是两个$1 \times 1$的卷积层
    * 这两个$1 \times 1$卷积层充当带有ReLU激活函数的逐像素全连接层
* 从另一个角度看，即将空间维度中的每个像素视为单个样本，将通道维度视为不同特征（feature）

In [1]:
import torch
from d2l import torch as d2l
from torch import nn


def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
    )

## NiN模型

<center><img src="../img/nin.svg" width="50%"></center>
<center>对比 VGG 和 NiN 及它们的块之间主要架构差异</center>

## NiN模型

* NiN使用窗口形状为$11\times 11$、$5\times 5$和$3\times 3$的卷积层，输出通道数量与AlexNet中的相同
* 每个NiN块后有一个最大池化层
    * 池化窗口形状为$3\times 3$，步幅为2
* 完全取消了全连接层
    * 使用一个NiN块，其输出通道数等于标签类别的数量
    * *全局平均汇聚层*（global average pooling layer）

阅读：网络中的网络\_NiN.pdf

In [2]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten(),
)

查看每个块的输出形状

In [3]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "output shape:\t", X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])
